# My New Jupyter Notebook

In [ ]:
print('Hello, world!')

## Add Cells to Notebook
In this section, we will add different types of cells (code, markdown) to the notebook.

### This is a Markdown Cell

In [ ]:
a = 10
b = 20
a + b

### Another Markdown Cell

In [ ]:
import math
math.sqrt(16)